In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON .. && make -j`nproc`
!mkdir openpose/images openpose/videos 

[ 17%] Building NVCC (Device) object src/caffe/CMakeFiles/cuda_compile_1.dir/layers/cuda_compile_1_generated_im2col_layer.cu.o
[ 17%] Building NVCC (Device) object src/caffe/CMakeFiles/cuda_compile_1.dir/layers/cuda_compile_1_generated_inner_product_layer.cu.o
[ 17%] Building NVCC (Device) object src/caffe/CMakeFiles/cuda_compile_1.dir/layers/cuda_compile_1_generated_log_layer.cu.o
[ 19%] Building NVCC (Device) object src/caffe/CMakeFiles/cuda_compile_1.dir/layers/cuda_compile_1_generated_lrn_layer.cu.o
[ 19%] Building NVCC (Device) object src/caffe/CMakeFiles/cuda_compile_1.dir/layers/cuda_compile_1_generated_lstm_unit_layer.cu.o
[ 19%] Building NVCC (Device) object src/caffe/CMakeFiles/cuda_compile_1.dir/layers/cuda_compile_1_generated_mvn_layer.cu.o
[ 20%] Building NVCC (Device) object src/caffe/CMakeFiles/cuda_compile_1.dir/layers/cuda_compile_1_generated_pooling_layer.cu.o
[ 20%] Building NVCC (Device) object src/caffe/CMakeFiles/cuda_compile_1.dir/layers/cuda_compile_1_generated_

In [3]:
%%sh
/content/openpose/models/getModels.sh

--2022-08-06 11:45:52--  http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/body_25/pose_iter_584000.caffemodel
Resolving posefs1.perception.cs.cmu.edu (posefs1.perception.cs.cmu.edu)... 128.2.220.57
Connecting to posefs1.perception.cs.cmu.edu (posefs1.perception.cs.cmu.edu)|128.2.220.57|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104715850 (100M) [text/plain]
Saving to: ‘pose/body_25/pose_iter_584000.caffemodel’

     0K .......... .......... .......... .......... ..........  0%  983K 1m44s
    50K .......... .......... .......... .......... ..........  0%  984K 1m44s
   100K .......... .......... .......... .......... ..........  0% 1.91M 87s
   150K .......... .......... .......... .......... ..........  0% 74.5M 65s
   200K .......... .......... .......... .......... ..........  0% 1.93M 62s
   250K .......... .......... .......... .......... ..........  0% 1.94M 61s
   300K .......... .......... .......... .......... ..........  0%  120M 52s
   

In [19]:
from google.colab.patches import cv2_imshow
import cv2
import csv
import glob 
import os 

In [27]:
def output_keypoints(frame, proto_file, weights_file, threshold, model_name, BODY_PARTS):
    global points

    # 네트워크 불러오기
    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)

    # 입력 이미지의 사이즈 정의
    image_height = 368
    image_width = 368

    # 네트워크에 넣기 위한 전처리
    input_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (image_width, image_height), (0, 0, 0), swapRB=False, crop=False)

    # 전처리된 blob 네트워크에 입력
    net.setInput(input_blob)

    # 결과 받아오기
    out = net.forward()
    # The output is a 4D matrix :
    # The first dimension being the image ID ( in case you pass more than one image to the network ).
    # The second dimension indicates the index of a keypoint.
    # The model produces Confidence Maps and Part Affinity maps which are all concatenated.
    # For COCO model it consists of 57 parts – 18 keypoint confidence Maps + 1 background + 19*2 Part Affinity Maps. Similarly, for MPI, it produces 44 points.
    # We will be using only the first few points which correspond to Keypoints.
    # The third dimension is the height of the output map.
    out_height = out.shape[2]
    # The fourth dimension is the width of the output map.
    out_width = out.shape[3]

    # 원본 이미지의 높이, 너비를 받아오기
    frame_height, frame_width = frame.shape[:2]

    # 포인트 리스트 초기화
    points = []

    #print(f"\n============================== {model_name} Model ==============================")
    for i in range(len(BODY_PARTS)):

        # 신체 부위의 confidence map
        prob_map = out[0, i, :, :]

        # 최소값, 최대값, 최소값 위치, 최대값 위치
        min_val, prob, min_loc, point = cv2.minMaxLoc(prob_map)

        # 원본 이미지에 맞게 포인트 위치 조정
        x = (frame_width * point[0]) / out_width
        x = int(x)
        y = (frame_height * point[1]) / out_height
        y = int(y)

        if prob > threshold:  # [pointed]
            #cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            #cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, lineType=cv2.LINE_AA)

            points.append((x, y))
            if i==1 or i==2 or i==3or i==4or i==5or i==6 or i==7 or i==8 or i==9or i==10 or i==11 or i==12 or i==13 or i==14 or i==19or i==20or i==21or i==22 or i==23 or i==24:
              wr.writerow([BODY_PARTS[i],x,y])
            #print(f"[pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

        else:  # [not pointed]
            #cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            #cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)

            points.append(None)
            if i==1 or i==2 or i==3or i==4or i==5or i==6 or i==7 or i==8 or i==9or i==10 or i==11 or i==12 or i==13 or i==14 or i==19or i==20or i==21or i==22 or i==23 or i==24:
              wr.writerow([BODY_PARTS[i],None,None])
            #print(f"[not pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

    #cv2_imshow("Output_Keypoints", frame)
    #cv2_imshow(frame)
    cv2.waitKey(0)
    return frame

In [5]:
def output_keypoints_with_lines(frame, POSE_PAIRS):
    print()
    for pair in POSE_PAIRS:
        part_a = pair[0]  # 0 (Head)
        part_b = pair[1]  # 1 (Neck)
        if points[part_a] and points[part_b]:
            print(f"[linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")
            cv2.line(frame, points[part_a], points[part_b], (0, 255, 0), 3)
        else:
            print(f"[not linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")

    #cv2_imshow("output_keypoints_with_lines", frame)
    cv2_imshow(frame)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    

In [6]:
BODY_PARTS_BODY_25 = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                      5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "MidHip", 9: "RHip",
                      10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee", 14: "LAnkle",
                      15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe",
                      20: "LSmallToe", 21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel", 25: "Background"}

POSE_PAIRS_BODY_25 = [[0, 1], [0, 15], [0, 16], [1, 2], [1, 5], [1, 8], [8, 9], [8, 12], [9, 10], [12, 13], [2, 3],
                      [3, 4], [5, 6], [6, 7], [10, 11], [13, 14], [15, 17], [16, 18], [14, 21], [19, 21], [20, 21],
                      [11, 24], [22, 24], [23, 24]]

In [28]:
protoFile_body_25 = "/content/openpose/models/pose/body_25/pose_deploy.prototxt"
weightsFile_body_25 = "/content/openpose/models/pose/body_25/pose_iter_584000.caffemodel"

# 이미지 경로
man =glob.glob('/content/gdrive/MyDrive/img/*.jpg')

print(man)
# 키포인트를 저장할 빈 리스트
points = []

# 이미지 읽어오기

f=open('Lstm.csv','w',encoding='utf-8',newline='')
wr=csv.writer(f)
for i in man:
  wr.writerow([i])
  frame_mpii = cv2.imread(i)
  frame_body_25 = frame_mpii.copy()
  frame_BODY_25 = output_keypoints(frame=frame_body_25, proto_file=protoFile_body_25, weights_file=weightsFile_body_25,threshold=0.2, model_name="BODY_25", BODY_PARTS=BODY_PARTS_BODY_25)
#output_keypoints_with_lines(frame=frame_BODY_25, POSE_PAIRS=POSE_PAIRS_BODY_25)
f.close()

['/content/gdrive/MyDrive/img/0.jpg', '/content/gdrive/MyDrive/img/2.jpg', '/content/gdrive/MyDrive/img/4.jpg', '/content/gdrive/MyDrive/img/7.jpg', '/content/gdrive/MyDrive/img/5.jpg', '/content/gdrive/MyDrive/img/3.jpg', '/content/gdrive/MyDrive/img/11.jpg', '/content/gdrive/MyDrive/img/14.jpg', '/content/gdrive/MyDrive/img/1.jpg', '/content/gdrive/MyDrive/img/9.jpg', '/content/gdrive/MyDrive/img/16.jpg', '/content/gdrive/MyDrive/img/6.jpg', '/content/gdrive/MyDrive/img/19.jpg', '/content/gdrive/MyDrive/img/21.jpg', '/content/gdrive/MyDrive/img/8.jpg', '/content/gdrive/MyDrive/img/28.jpg', '/content/gdrive/MyDrive/img/26.jpg', '/content/gdrive/MyDrive/img/10.jpg', '/content/gdrive/MyDrive/img/17.jpg', '/content/gdrive/MyDrive/img/18.jpg', '/content/gdrive/MyDrive/img/33.jpg', '/content/gdrive/MyDrive/img/35.jpg', '/content/gdrive/MyDrive/img/20.jpg', '/content/gdrive/MyDrive/img/22.jpg', '/content/gdrive/MyDrive/img/23.jpg', '/content/gdrive/MyDrive/img/13.jpg', '/content/gdrive/MyDr

In [24]:
files =glob.glob('/content/gdrive/MyDrive/img/*.jpg')
print(files)

['/content/gdrive/MyDrive/img/0.jpg', '/content/gdrive/MyDrive/img/2.jpg', '/content/gdrive/MyDrive/img/4.jpg', '/content/gdrive/MyDrive/img/7.jpg', '/content/gdrive/MyDrive/img/5.jpg', '/content/gdrive/MyDrive/img/3.jpg', '/content/gdrive/MyDrive/img/11.jpg', '/content/gdrive/MyDrive/img/14.jpg', '/content/gdrive/MyDrive/img/1.jpg', '/content/gdrive/MyDrive/img/9.jpg', '/content/gdrive/MyDrive/img/16.jpg', '/content/gdrive/MyDrive/img/6.jpg', '/content/gdrive/MyDrive/img/19.jpg', '/content/gdrive/MyDrive/img/21.jpg', '/content/gdrive/MyDrive/img/8.jpg', '/content/gdrive/MyDrive/img/28.jpg', '/content/gdrive/MyDrive/img/26.jpg', '/content/gdrive/MyDrive/img/10.jpg', '/content/gdrive/MyDrive/img/17.jpg', '/content/gdrive/MyDrive/img/18.jpg', '/content/gdrive/MyDrive/img/33.jpg', '/content/gdrive/MyDrive/img/35.jpg', '/content/gdrive/MyDrive/img/20.jpg', '/content/gdrive/MyDrive/img/22.jpg', '/content/gdrive/MyDrive/img/23.jpg', '/content/gdrive/MyDrive/img/13.jpg', '/content/gdrive/MyDr